# MAVERICK - Modell Tanítás

Ez a notebook a hierarchikus CNN modell tanítását végzi Google Colab-ban.

In [ ]:
# Google Drive csatlakoztatás
from google.colab import drive
drive.mount('/content/drive')

# Projekt klónozása
!git clone https://github.com/Koppi02/AI-Agent.git
%cd AI-Agent

# Telepítés
!pip install -q -r requirements.txt

In [ ]:
import sys
sys.path.insert(0, '/content/AI-Agent')

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from src.data.loader import DataLoader
from src.data.preprocessor import ImagePreprocessor
from src.models.cnn_model import build_hierarchical_cnn, compile_model
from src.utils.io_utils import load_config, ensure_dir
from src.utils.visualization import plot_training_history

In [ ]:
# Konfiguráció
config = load_config('config/config.yaml')
config['data']['drive_path'] = '/content/drive/MyDrive/AI_PROJECTS/'

# GPU ellenõrzés
print(f"GPU elérhetõ: {tf.config.list_physical_devices('GPU')}")

## 1. Adatok Betöltése

In [ ]:
data_loader = DataLoader(config)
data_loader.load_abs_torveny()
data_loader.load_labels(use_processed=True)

## 2. Képek Elõkészítése

In [ ]:
preprocessor = ImagePreprocessor(config, data_loader.labels_df)
preprocessor.preprocess_labels()
train_dataset, val_dataset = preprocessor.create_datasets()

## 3. Modell Építése

In [ ]:
num_segments, num_brands, num_basebrands, num_advertisers = data_loader.get_num_categories()

model = build_hierarchical_cnn(
    image_shape=(512, 512, 3),
    num_segments=num_segments,
    num_brands=num_brands,
    num_basebrands=num_basebrands,
    num_advertisers=num_advertisers
)

compile_model(model, learning_rate=0.0001)
model.summary()

## 4. Tanítás

In [ ]:
# Callbacks
checkpoint_dir = config['checkpoints']['directory']
ensure_dir(checkpoint_dir)

callbacks = [
    ModelCheckpoint(
        filepath=checkpoint_dir + "cp-{epoch:04d}.weights.h5",
        save_weights_only=True,
        verbose=1
    ),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [ ]:
# Tanítás
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

## 5. Eredmények Vizualizációja

In [ ]:
plot_training_history(history.history)

## 6. Modell Mentése

In [ ]:
model_save_path = config['checkpoints']['model_save_path']
model.save(model_save_path)
print(f"Modell elmentve: {model_save_path}")